In [1]:
import random
import os
import numpy as np
import torch
import pickle

from os.path import basename, join
from glob import glob


from scipy.io import wavfile,loadmat
from scipy.io.wavfile import read
from glob import glob
from os.path import join,basename
from pydub import AudioSegment

from tqdm import tqdm
from nnmnkwii import metrics

import layers
from scipy.spatial.distance  import euclidean
from scipy import signal,stats,fftpack
import statistics

In [2]:
def load_wav_to_torch(path):
    sampling_rate, data = read(path)
    return torch.FloatTensor(data.astype(np.float32)), sampling_rate

In [3]:
def compute_melspec(data):
    audio_norm = data/32768
    audio_norm = audio_norm.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    stft=layers.TacotronSTFT(1024, 200, 800,80, 16000, 0.0,8000.0)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = torch.squeeze(melspec, 0)
    return melspec

In [4]:
def get_spec(mel):
    stft = layers.TacotronSTFT(1024,200,800, sampling_rate=16000)
    mel_decompress = stft.spectral_de_normalize(mel)
    mel_decompress = mel_decompress.transpose(1, 2).data.cpu()
    mel_decompress = mel_decompress.float()
    spec_from_mel_scaling = 1000
    spec_from_mel = torch.mm(mel_decompress[0], stft.mel_basis)
    spec_from_mel = spec_from_mel.transpose(0, 1).unsqueeze(0)
    spec_from_mel = spec_from_mel * spec_from_mel_scaling
    return spec_from_mel

In [5]:
def load_dataset(filename, split="|"):
    with open(filename, encoding='utf-8') as f:
        dataset = [line.strip().split(split) for line in f]
    return dataset

In [6]:
def create_speaker_lookup_table(dataset):
        speaker_ids = np.sort(np.unique([x[2] for x in dataset]))
        d = {int(speaker_ids[i]): i for i in range(len(speaker_ids))}
        return d

In [7]:
train_path =  '/home/drobo/hironishi/work/research/data_manage/tacotron2_dataset/csj_dataset/txts/002/train.txt'
dataset = load_dataset(train_path)
#Speaker identification for input to the model
ISMS = create_speaker_lookup_table(dataset) # Input Speaker_id :  Speaker_id in Model

In [8]:
base_dir = './DATA/005'
os.makedirs(base_dir,exist_ok=True)
#標本化周波数
fs =16000
conpute = True

In [9]:
Grand_truth_dir = join(base_dir,'Grand_truth')
os.makedirs(Grand_truth_dir,exist_ok=True)
tool_dir = join(base_dir,'tool')
os.makedirs(tool_dir,exist_ok=True)
SSI_path = join(tool_dir,'SSI.pickle')
synth_text_path = join(tool_dir,'exp_text.txt')
source_list_path = join(tool_dir,'source_list')

In [10]:
test_path ='/home/drobo/hironishi/work/research/data_manage/tacotron2_dataset/csj_dataset/txts/002/test.txt'
with open(test_path,'r') as f:
    # Audio path:Text:Speaker_id
    FATS = {} 
    DATS = {}
    for line in f:
        audio_path,text,speaker = line.split('|')
        speaker=int(speaker.replace('\n','')) 
        #発話の長さが10字以上のもの   
        if len(text)>=10 and audio_path.split('/')[-3]=='WAV':
            FATS[audio_path]={}
            FATS[audio_path][text]=speaker
        if len(text)>=10 and audio_path.split('/')[-3]=='WAV_df_fps':
            DATS[audio_path]={}
            DATS[audio_path][text]=speaker

In [11]:
FAkeys = random.sample(list(FATS.keys()),100)
DAkeys = random.sample(list(DATS.keys()),100)

In [13]:
if conpute:
    with open(SSI_path,mode ='wb')as out:
        SSI_converted = [ ISMS[list(DATS[audio].values())[0]]  for audio in DAkeys ]
        pickle.dump(SSI_converted,out)
        out.close()
    
    for i, file in enumerate(DAkeys):
            %cp $file $Grand_truth_dir
            f_name = os.path.basename(file)
            os.rename(f"{Grand_truth_dir+'/'}{f_name}", f"{Grand_truth_dir+'/'}{'text_'+str(i).zfill(3)}.wav")
    
    with open(synth_text_path,'w') as syn:
        with open(source_list_path,'w') as w:
            for path in DAkeys:
                text = list(DATS[path].keys())[0]+'\n'
                path = path+'\n'
                syn.write(text)
                w.write(path) 
    
    from audio_processing import griffin_lim
    from scipy.io.wavfile  import write
    with open(join(base_dir,'tool/source_list'),'r')as f:
        wavpaths =  [path.replace('\n','')  for path in f]
        
    gt_mels = []  
    for wavpath in wavpaths:
        data,sr = load_wav_to_torch(wavpath)
        mel_spec = compute_melspec(data)
        gt_mels.append(mel_spec)
        
        
    out_dir = join(base_dir,'Grand_truth_griffin_lim')
    os.makedirs(out_dir,exist_ok=True)
    stft = layers.TacotronSTFT(1024,200,800, sampling_rate=16000)
    for i, mel in tqdm(enumerate(gt_mels)):
        spec = get_spec(mel.unsqueeze(0))
        audio = griffin_lim(torch.autograd.Variable(spec[:, :, :-1]), stft.stft_fn, 100)
        audio = audio.squeeze()
        audio = audio.data.cpu().numpy()
        #audio = audio/np.max(np.abs(audio))
        write(join(out_dir,'text_{}.wav'.format(str(i).zfill(3))),16000,audio)

100it [01:51,  1.11s/it]
